<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/extract_data/Extract_Standardised_Precipitation_Evapotranspiration_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# Pandas allows us to create a spreadsheet output
# datasets
import pandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/burnt_veg/burnt_area_roi.geojson"


# Dates before the fire
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2024, month=5, day=28)

# No Data Value
out_no_data_val = 0.0

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
gee_bbox_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
roi_ee_poly = pb_gee_tools.convert_types.convert_vector_to_gee_polygon(vec_file = vec_roi_file)

In [ ]:
spei_img_col = ee.ImageCollection('CSIC/SPEI/2_10').filterBounds(gee_bbox_aoi).filter(ee.Filter.date(start_date, end_date))

In [ ]:
def create_time_series(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=roi_ee_poly)
    data_lut = {"date": image.date().format()}
    for mnth in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48"]:
        data_lut[f"SPEI_{mnth}_month"] = stats.get(f"SPEI_{mnth}_month")

    return ee.Feature(None, data_lut)

In [ ]:
spei_time_series = spei_img_col.map(create_time_series).getInfo()

In [ ]:
# Convert time series to a pandas DataFrame
spei_feats = spei_time_series["features"]

# Create a list of the dates and temperature data
spei_data = list()
for spei_feat in spei_feats:
    if "date" in spei_feat["properties"]:
        data_lut = {"date": spei_feat["properties"]["date"]}
        for mnth in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48"]:
            data_lut[f"SPEI_{mnth}_month"] = spei_feat["properties"][f"SPEI_{mnth}_month"]
        spei_data.append(data_lut)

# Convert the list of a Pandas Dataframe
spei_df = pandas.DataFrame(spei_data)
# Convert the date column to a datatime object
spei_df["date"] = pandas.to_datetime(spei_df["date"])
# Sort the Dataframe by the date
spei_df = spei_df.sort_values("date")
spei_df

In [ ]:
# Save the pandas dataframes to the google drive
spei_df.to_excel(
    "/content/drive/MyDrive/burnt_veg/burnt_area_Standardised_Precipitation_Evapotranspiration_Index_data.xlsx", index=False
)
